In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
from csdid.att_gt import ATTgt

In [257]:
cleaned_bart_df = pd.read_stata('/Users/austincoffelt/Documents/bart_fare_gates_project/data/bart_ridership_2023_2025_with_treatment.dta')

# Display the first few rows
print(cleaned_bart_df.shape)
cleaned_bart_df.head()

(2362500, 52)


,origin,destination,route,date,dow,riders,log_riders,origin_completion_date,destination_completion_date,origin_start_date,...,destination_airport,treated_origin,treated_destination,treated_either,treated_both,under_construction,earliest_completion_route,time_to_treatment_route,stops,transfer_needed
0,RICH,12TH,1901.0,2025-02-09,0.0,36.0,3.583519,2024-09-13,2025-02-04,2024-09-03,...,0,1.0,1.0,1.0,1.0,0.0,23632.0,149.0,8.0,No transfer
1,MLBR,12TH,1301.0,2023-09-30,6.0,19.0,2.944439,2025-07-16,2025-02-04,2025-06-16,...,0,0.0,0.0,0.0,0.0,0.0,23776.0,-493.0,15.0,No transfer
2,ROCK,12TH,1951.0,2025-07-07,1.0,82.0,4.406719,2025-03-10,2025-02-04,2025-02-14,...,0,1.0,1.0,1.0,1.0,0.0,23776.0,153.0,3.0,No transfer
3,NCON,12TH,1501.0,2024-11-15,5.0,19.0,2.944439,2025-05-19,2025-02-04,2025-04-30,...,0,0.0,0.0,0.0,0.0,0.0,23776.0,-81.0,9.0,No transfer
4,ROCK,12TH,1951.0,2024-03-29,5.0,52.0,3.951244,2025-03-10,2025-02-04,2025-02-14,...,0,0.0,0.0,0.0,0.0,0.0,23776.0,-312.0,3.0,No transfer


In [258]:
# create no parking variable
cleaned_bart_df['origin_no_parking'] = cleaned_bart_df['origin_parking'] == 0
cleaned_bart_df['destination_no_parking'] = cleaned_bart_df['destination_parking'] == 0

cleaned_bart_df['no_parking'] = (
    (cleaned_bart_df['origin_no_parking'] | cleaned_bart_df['destination_no_parking'])
    .astype(int))

# make date variables to datetime
cleaned_bart_df['date'] = pd.to_datetime(cleaned_bart_df['date'])
cleaned_bart_df['origin_completion_date'] = pd.to_datetime(cleaned_bart_df['origin_completion_date'])
cleaned_bart_df['destination_completion_date'] = pd.to_datetime(cleaned_bart_df['destination_completion_date'])
cleaned_bart_df['origin_start_date'] = pd.to_datetime(cleaned_bart_df['origin_start_date'])
cleaned_bart_df['destination_start_date'] = pd.to_datetime(cleaned_bart_df['destination_start_date'])

cleaned_bart_df.head()

,origin,destination,route,date,dow,riders,log_riders,origin_completion_date,destination_completion_date,origin_start_date,...,treated_either,treated_both,under_construction,earliest_completion_route,time_to_treatment_route,stops,transfer_needed,origin_no_parking,destination_no_parking,no_parking
0,RICH,12TH,1901.0,2025-02-09,0.0,36.0,3.583519,2024-09-13,2025-02-04,2024-09-03,...,1.0,1.0,0.0,23632.0,149.0,8.0,No transfer,False,True,1
1,MLBR,12TH,1301.0,2023-09-30,6.0,19.0,2.944439,2025-07-16,2025-02-04,2025-06-16,...,0.0,0.0,0.0,23776.0,-493.0,15.0,No transfer,False,True,1
2,ROCK,12TH,1951.0,2025-07-07,1.0,82.0,4.406719,2025-03-10,2025-02-04,2025-02-14,...,1.0,1.0,0.0,23776.0,153.0,3.0,No transfer,False,True,1
3,NCON,12TH,1501.0,2024-11-15,5.0,19.0,2.944439,2025-05-19,2025-02-04,2025-04-30,...,0.0,0.0,0.0,23776.0,-81.0,9.0,No transfer,False,True,1
4,ROCK,12TH,1951.0,2024-03-29,5.0,52.0,3.951244,2025-03-10,2025-02-04,2025-02-14,...,0.0,0.0,0.0,23776.0,-312.0,3.0,No transfer,False,True,1


In [259]:
# clean data for python
cleaned_bart_df['route'] = cleaned_bart_df['origin'] + ' | ' + cleaned_bart_df['destination']
cleaned_bart_df['earliest_completion_route'] = cleaned_bart_df[['origin_completion_date', 'destination_completion_date']].min(axis=1)

cleaned_bart_df.head()

,origin,destination,route,date,dow,riders,log_riders,origin_completion_date,destination_completion_date,origin_start_date,...,treated_either,treated_both,under_construction,earliest_completion_route,time_to_treatment_route,stops,transfer_needed,origin_no_parking,destination_no_parking,no_parking
0,RICH,12TH,RICH | 12TH,2025-02-09,0.0,36.0,3.583519,2024-09-13,2025-02-04,2024-09-03,...,1.0,1.0,0.0,2024-09-13,149.0,8.0,No transfer,False,True,1
1,MLBR,12TH,MLBR | 12TH,2023-09-30,6.0,19.0,2.944439,2025-07-16,2025-02-04,2025-06-16,...,0.0,0.0,0.0,2025-02-04,-493.0,15.0,No transfer,False,True,1
2,ROCK,12TH,ROCK | 12TH,2025-07-07,1.0,82.0,4.406719,2025-03-10,2025-02-04,2025-02-14,...,1.0,1.0,0.0,2025-02-04,153.0,3.0,No transfer,False,True,1
3,NCON,12TH,NCON | 12TH,2024-11-15,5.0,19.0,2.944439,2025-05-19,2025-02-04,2025-04-30,...,0.0,0.0,0.0,2025-02-04,-81.0,9.0,No transfer,False,True,1
4,ROCK,12TH,ROCK | 12TH,2024-03-29,5.0,52.0,3.951244,2025-03-10,2025-02-04,2025-02-14,...,0.0,0.0,0.0,2025-02-04,-312.0,3.0,No transfer,False,True,1


Here is how I plan on grouping treatment dates for easier computation:
- group 1: WOAK
- group 2: FRVL and CIVC
- group 3: RICH and 24TH
- group 4: ANTC
- group 5: 16TH
- group 6: SFIA and COLS
- group 7: POWL
- group 8: WARM
- group 9: MONT, DUBL, HAYW
- group 10: SHAY
- group 11: 12TH and EMBR
- group 12: ORIN and DBRK
- group 13: LAFY, PCTR, and ROCK
- group 14: SANL and GLEN
- group 15: PLZA and CONC
- group 16: DELN, MCAR, LAKE
- group 17: DALY, PITT, NCON, WCRK, NBRK, and PHIL
- group 18: ASHB
- group 19: BAYF, COLM, and 19th
- group 20: MLBR
- group 21: the rest (untreated)

In [261]:
# Group together time periods for easier computation

# group 1 (Start 12/2023 not 1/2023)
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2023-11-30')]

# group 2
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2024-8-28') | (cleaned_bart_df['date'] < '2024-8-21')]

# group 3
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2024-9-20') | (cleaned_bart_df['date'] < '2024-9-13')]

# group 4 (no change)

# group 5 (no change)

# group 6
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2024-10-31') | (cleaned_bart_df['date'] < '2024-10-26')]

# group 7 (no change)

# group 8 (no change)

# group 9 
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2024-12-20') | (cleaned_bart_df['date'] < '2024-12-18')]

# group 10 (no change)

# group 11 
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-2-11') | (cleaned_bart_df['date'] < '2025-2-04')]

# group 12
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-2-24') | (cleaned_bart_df['date'] < '2025-2-22')]

# group 13
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-3-09') | (cleaned_bart_df['date'] < '2025-3-07')]

# group 14 (no change)

# group 15 (no change)

# group 16
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-4-29') | (cleaned_bart_df['date'] < '2025-4-24')]

# group 17
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-5-22') | (cleaned_bart_df['date'] < '2025-5-18')]

# group 18 (no change)

# group 19
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] > '2025-6-18') | (cleaned_bart_df['date'] < '2025-6-15')]

# group 20 (no change)

# group 21
cleaned_bart_df = cleaned_bart_df[(cleaned_bart_df['date'] < '2025-7-25')]

cleaned_bart_df.shape

(1370000, 55)

In [263]:
# Check if panel is balanced
panel_check = cleaned_bart_df.groupby('route')['date'].nunique()
total_periods = cleaned_bart_df['date'].nunique()

print(f"Total time periods: {total_periods}")
print(f"Number of routes: {len(panel_check)}")
print(f"Routes with complete data: {(panel_check == total_periods).sum()}")
print(f"Routes with missing periods: {(panel_check < total_periods).sum()}")
print(f"Percentage balanced: {(panel_check == total_periods).mean():.2%}")

# Show distribution of observations per route
print("\nObservations per route:")
print(panel_check.describe())

Total time periods: 548
Number of routes: 2500
Routes with complete data: 2500
Routes with missing periods: 0
Percentage balanced: 100.00%

Observations per route:
count    2500.0
mean      548.0
std         0.0
min       548.0
25%       548.0
50%       548.0
75%       548.0
max       548.0
Name: date, dtype: float64


In [266]:
# preperation for model
# Find the absolute earliest date in the entire dataset to use as a reference point.
min_date = cleaned_bart_df['date'].min()

# Convert both columns to the number of days since the minimum date
cleaned_bart_df['date_numeric'] = (cleaned_bart_df['date'] - min_date).dt.days
cleaned_bart_df['earliest_completion_route_numeric'] = (cleaned_bart_df['earliest_completion_route'] - min_date).dt.days

# Fill the missing values (NaT) in the treatment date with 0 for never-treated units
cleaned_bart_df['earliest_completion_route_numeric'] = cleaned_bart_df['earliest_completion_route_numeric'].fillna(0).astype(int)

In [269]:
model1 = ATTgt(yname = "log_riders",
               gname = "earliest_completion_route_numeric",
               idname = "route",           
               tname = "date_numeric",             
               xformla = None,
               data = cleaned_bart_df,
               control_group="notyettreated",
               biters = 10).fit()

model1.summ_attgt().summary2

dropped, 53645, rows from original data due to missing data


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'